In [6]:
import pandas as pd
import json
import sqlite3

In [7]:
orders = pd.read_csv("C:/Users/sjcet/Downloads/orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [8]:
with open("C:/Users/sjcet/Downloads/users.json", "r") as f:
    users = pd.json_normalize(json.load(f))
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [9]:
conn = sqlite3.connect(":memory:")

with open("C:/Users/sjcet/Downloads/restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [10]:
df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [11]:
df.shape


(10000, 12)

In [12]:
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)
df["quarter"] = df["order_date"].dt.to_period("Q")


In [13]:
df[df["membership"]=="Gold"] \
.groupby("city")["total_amount"].sum().idxmax()


'Chennai'

In [14]:
df.groupby("cuisine")["total_amount"].mean().idxmax()


'Mexican'

In [15]:
user_spend = df.groupby("user_id")["total_amount"].sum()
user_spend[user_spend > 1000].count()


np.int64(2544)

In [16]:
df["rating_range"] = pd.cut(df["rating"], [3,3.5,4,4.5,5])
df.groupby("rating_range")["total_amount"].sum().idxmax()


C:\Users\sjcet\AppData\Local\Temp\ipykernel_17768\3135384773.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum().idxmax()


Interval(4.5, 5.0, closed='right')

In [17]:
df[df["membership"]=="Gold"] \
.groupby("city")["total_amount"].mean().idxmax()


'Chennai'

In [19]:
df.groupby("cuisine")["restaurant_id"].nunique().idxmin()


'Chinese'

In [20]:
round((df[df["membership"]=="Gold"].shape[0] / df.shape[0]) * 100)


50

In [21]:
restaurant_stats = df.groupby("restaurant_name_x").agg(
    avg_order=("total_amount","mean"),
    order_count=("order_id","count")
)

restaurant_stats[restaurant_stats["order_count"] < 20] \
.sort_values("avg_order", ascending=False).head()


,avg_order,order_count
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19


In [22]:
df.groupby(["membership","cuisine"])["total_amount"].sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [23]:
df.groupby("quarter")["total_amount"].sum().idxmax()


Period('2023Q3', 'Q-DEC')

In [24]:
df[df["membership"]=="Gold"].shape[0]


4987

In [25]:
round(df[df["city"]=="Hyderabad"]["total_amount"].sum())


1889367

In [26]:
df["user_id"].nunique()


2883

In [27]:
round(df[df["membership"]=="Gold"]["total_amount"].mean(), 2)


np.float64(797.15)

In [28]:
df[df["rating"] >= 4.5].shape[0]


3374

In [30]:
top_gold_city = df[df["membership"]=="Gold"] \
.groupby("city")["total_amount"].sum().idxmax()

df[(df["membership"]=="Gold") & (df["city"]==top_gold_city)].shape[0]


1337